# 1: Импорт и базовые функции

In [1]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import re

<h4> Функция для выбора годов:

In [2]:
def select_years(driver, years):
    for year in years:
        try:
            checkbox = driver.find_element(By.ID, f"years_{year}")
            if not checkbox.is_selected():
                checkbox.click()
        except Exception as e:
            print(f"Не удалось выбрать год {year}: {e}")

In [3]:
# Настройки
LOGIN = "Trokzen"
PASSWORD = "R1o2b3e4"
ORG_ID = "14812"
MAX_PAGES = 3
OUTPUT_DIR = "elibrary_data"

os.makedirs(OUTPUT_DIR, exist_ok=True)

def initialize_browser():
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(options=options)
    return driver

def login_elibrary(driver):
    print("Начало авторизации...")
    driver.get("https://www.elibrary.ru/")
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.ID, "login"))
        ).send_keys(LOGIN)
        driver.find_element(By.ID, "password").send_keys(PASSWORD)
        driver.find_element(By.XPATH, "//div[@class='butred' and contains(text(), 'Вход')]").click()
        print("Авторизация успешна")
    except Exception as e:
        print(f"Ошибка авторизации: {e}")
        driver.quit()
        raise

def solve_captcha(driver):
    """Пауза для ручного решения капчи, если она появилась."""
    try:
        WebDriverWait(driver, 5).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//iframe[contains(@title, 'тест Тьюринга')]")))
        print("Обнаружена капча! Решите её вручную в браузере и нажмите Enter здесь.")
        input("Нажмите Enter после решения капчи...")
        driver.switch_to.default_content()
        return True
    except:
        driver.switch_to.default_content()
        return False

def extract_year(text):
    match = re.search(r'\b(19|20)\d{2}\b', text)
    if match:
        return int(match.group(0))
    return None

def parse_page(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    articles = []
    table = soup.find('table', {'id': 'restab'})
    if not table:
        return articles
    for row in table.find_all('tr')[2:]:
        cols = row.find_all('td')
        if len(cols) < 2:
            continue
        title_link = cols[1].find('a', href=True)
        if not title_link or 'id=' not in title_link['href']:
            continue
        pub_id = title_link['href'].split('id=')[1].split('&')[0]
        title = title_link.get_text(strip=True)
        authors = cols[1].find('i').get_text(strip=True) if cols[1].find('i') else "Нет авторов"
        
        # --- Новый код для поиска года ---
        year_pub = None
        font_tags = cols[1].find_all('font')
        for font in font_tags:
            year_pub = extract_year(font.get_text(" ", strip=True))
            if year_pub:
                break
        if not year_pub:
            year_pub = extract_year(cols[1].get_text(" ", strip=True))
        # --- Конец нового кода ---

        articles.append({
            'ID': pub_id,
            'Название': title,
            'Авторы': authors,
            'Год публикации': year_pub if year_pub else ""
        })
    return articles

def parse_with_filter(driver, filter_value, filter_name):
    print(f"\nНачинаем парсинг с фильтром: {filter_name}")
    select = Select(WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "show_option"))))
    select.select_by_value(filter_value)
    articles = []
    page_num = 1
    while page_num <= MAX_PAGES:
        print(f"Обработка страницы {page_num}...")
        try:
            page_articles = parse_page(driver)
            if not page_articles:
                print("Не найдено статей на странице")
                break
            articles.extend(page_articles)
            if page_num % 5 == 0:
                pd.DataFrame(articles).to_csv(
                    f"{OUTPUT_DIR}/{filter_name}_page_{page_num}.csv",
                    index=False,
                    encoding='utf-8-sig'
                )
                print(f"Промежуточное сохранение: {len(articles)} статей")
        except Exception as e:
            print(f"Ошибка парсинга страницы {page_num}: {e}")
            break
        try:
            next_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, 
                "//a[@title='Следующая страница' and contains(@href, 'javascript:goto_page')]")))
            next_btn.click()
            solve_captcha(driver)
            page_num += 1
        except:
            print("Достигнут конец списка или ошибка перехода")
            break
    output_file = f"{OUTPUT_DIR}/{filter_name}_final.csv"
    pd.DataFrame(articles).to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"Завершено. Всего {len(articles)} статей сохранено в {output_file}")
    return articles

# 2: Парсинг всех публикаций

In [4]:
years_to_parse = ['2024', '2025']  # список нужных годов

driver = initialize_browser()
login_elibrary(driver)
driver.get(f"https://elibrary.ru/org_items.asp?orgsid={ORG_ID}")
solve_captcha(driver)

select_years(driver, years_to_parse)  # выбираем нужные года

# Нажимаем кнопку "Поиск"
search_btn = driver.find_element(By.XPATH, "//div[@class='butred' and contains(text(), 'Поиск')]")
search_btn.click()

# (Желательно дождаться загрузки результатов)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "restab"))
)

all_articles = parse_with_filter(driver, "1", "all_publications")
driver.quit()

Начало авторизации...
Авторизация успешна


ReadTimeoutError: HTTPConnectionPool(host='localhost', port=57830): Read timed out. (read timeout=120)

# 3: Парсинг публикаций РИНЦ

In [11]:
years_to_parse = ['2024', '2025']  # список нужных годов

driver = initialize_browser()
login_elibrary(driver)
driver.get(f"https://elibrary.ru/org_items.asp?orgsid={ORG_ID}")
solve_captcha(driver)

select_years(driver, years_to_parse)  # выбираем нужные года

# Нажимаем кнопку "Поиск"
search_btn = driver.find_element(By.XPATH, "//div[@class='butred' and contains(text(), 'Поиск')]")
search_btn.click()

# (Желательно дождаться загрузки результатов)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "restab"))
)

rinc_articles = parse_with_filter(driver, "0", "rinc_publications")
driver.quit()

Начало авторизации...
Авторизация успешна

Начинаем парсинг с фильтром: rinc_publications
Обработка страницы 1...
Обработка страницы 2...
Обработка страницы 3...
Обработка страницы 4...
Обработка страницы 5...
Промежуточное сохранение: 500 статей
Обработка страницы 6...
Обработка страницы 7...
Обработка страницы 8...
Обработка страницы 9...
Обработка страницы 10...
Промежуточное сохранение: 1000 статей
Обработка страницы 11...
Обработка страницы 12...
Обработка страницы 13...
Обработка страницы 14...
Обработка страницы 15...
Промежуточное сохранение: 1500 статей
Обработка страницы 16...
Обработка страницы 17...
Обработка страницы 18...
Обработка страницы 19...
Обработка страницы 20...
Промежуточное сохранение: 2000 статей
Обработка страницы 21...
Обработка страницы 22...
Обработка страницы 23...
Обработка страницы 24...
Обработка страницы 25...
Промежуточное сохранение: 2500 статей
Обработка страницы 26...
Обработка страницы 27...
Обработка страницы 28...
Обработка страницы 29...
Обраб

# 4: Сопоставление и итоговый файл

In [12]:
rinc_ids = {art['ID'] for art in rinc_articles}
final_data = []
for article in all_articles:
    article['В_РИНЦ'] = 'Да' if article['ID'] in rinc_ids else 'Нет'
    final_data.append(article)
final_df = pd.DataFrame(final_data)
final_df.to_csv(f"{OUTPUT_DIR}/final_result.csv", index=False, encoding='utf-8-sig')
final_df.head()

,ID,Название,Авторы,В_РИНЦ
0,80340461,СХЕМОТЕХНИКА ТЕЛЕКОММУНИКАЦИОННЫХ УСТРОЙСТВ,"Редкова Н.А., Кузина Е.И., Кузин П.И.",Да
1,80340463,ЭЛЕКТРОННЫЕ УСТРОЙСТВА,"Базир Г.И., Иванов С.Е., Редкова Н.А., Кузина ...",Да
2,80490396,ТЕОРЕТИЧЕСКИЕ ОСНОВЫ ФИЗИЧЕСКОЙ ПОДГОТОВКИ СЛУ...,"Федюк Н.С., Федоров А.В., Марин А.П., Андросов...",Да
3,80641447,СОВРЕМЕННЫЕ ТЕНДЕНЦИИ ИНЖЕНЕРНОГО ОБРАЗОВАНИЯ,Нет авторов,Да
4,80528783,СПОСОБ ПРОЕКТИРОВАНИЯ СТРУКТУРЫ ВОЛОКОННО-ОПТИ...,"Горай И.И., Журавлёв Д.А., Калайтанова Е.В., М...",Да
